In [1]:
#!pip install n2v
#!pip install albumentations

In [1]:
# We import all our dependencies.
from n2v.models import N2VConfig, N2V
import numpy as np
from csbdeep.utils import plot_history
from n2v.utils.n2v_utils import manipulate_val_data
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator
from matplotlib import pyplot as plt
import urllib
import os
import glob
import zipfile
import cv2
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import albumentations as albu
import json
from functools import partial

import ssl
ssl._create_default_https_context = ssl._create_unverified_context


2023-05-02 15:53:56.361230: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
INBREAST_PATH="/kaggle/input/mias-cbis-ddsm-inbreast/Mammographies/INBreast"
MIAS_PATH="/kaggle/input/mias-cbis-ddsm-inbreast/Mammographies/MIAS"
CBIS_DDSM_PATH="/kaggle/input/mias-cbis-ddsm-inbreast/Mammographies/CBIS-DDSM"

In [3]:
input_shape=(1600,1600)

In [4]:
def get_images_paths(path):
    images_path=[]
    for (filepath, _, files) in os.walk(path, topdown=False):
        for file in files:
            if file.endswith(".png") and "MASK" not in file:
                images_path.append(os.path.join(filepath,file))
    return images_path
    

inbreast_files = get_images_paths(INBREAST_PATH)
cbis_ddsm_files = get_images_paths(MIAS_PATH)
mias_files = get_images_paths(CBIS_DDSM_PATH)

all_files = np.concatenate((inbreast_files, cbis_ddsm_files, mias_files), axis=None)

np.random.shuffle(all_files)
ds_all_files = tf.data.Dataset.from_tensor_slices(all_files)

2023-05-02 15:54:01.819604: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-02 15:54:01.826179: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-02 15:54:01.826350: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
# a name used to identify the model
model_name = 'n2v_2D'
# the base directory in which our model will live
basedir = '/kaggle/working/models'

# We are now creating our network model.
model = N2V(None, model_name, basedir=basedir)

Loading network weights from 'weights_best.h5'.


In [6]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
   
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    img = np.array(image).astype("uint8")
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)



In [7]:
with open(os.path.join(INBREAST_PATH,"roi_images.json"),"r") as f:
    rois_inbreast = json.load(f)
    
with open(os.path.join(CBIS_DDSM_PATH,"roi_images.json"),"r") as f:
    rois_cbis_ddsm = json.load(f)
    
with open(os.path.join(MIAS_PATH,"roi_images.json"),"r") as f:
    rois_mias = json.load(f)

print(len(rois_inbreast), len(rois_cbis_ddsm), len(rois_mias))

343 3103 111


In [20]:
##########################################################################################
################################   TEST   ################################################
##########################################################################################

rois = {}
rois.update(rois_inbreast)
rois.update(rois_cbis_ddsm)
rois.update(rois_mias)
len(rois)

3557

In [16]:
# generate YOLO data
YOLO_DIR_IMAGES="/kaggle/working/yolo_data/images"
YOLO_DIR_LABELS="/kaggle/working/yolo_data/labels"

os.makedirs(YOLO_DIR_IMAGES, exist_ok=True)
os.makedirs(YOLO_DIR_LABELS, exist_ok=True)

datasets_rois = {"mias": rois_mias, "inbreast": rois_inbreast, "cbis_ddsm": rois_cbis_ddsm}

dataset_to_dir = {"mias": MIAS_PATH,"inbreast": INBREAST_PATH,"cbis_ddsm": CBIS_DDSM_PATH}

from collections import defaultdict

##Check how balanced are abnormalities
types_ab = defaultdict(int)
for dataset, rois in datasets_rois.items():
    for k, rois in rois.items():
        for roi in rois:
            types_ab[roi["type"]] += 1
types_ab

defaultdict(int,
            {'CIRC': 24,
             'MISC': 15,
             'ASYM': 15,
             'ARCH': 19,
             'SPIC': 19,
             'CALC': 27,
             'Calcification': 7142,
             'Spiculated Region': 14,
             'Mass': 1812,
             'Cluster': 27,
             'Assymetry': 1,
             'Asymmetry': 5,
             'Distortion': 3,
             'Calcifications': 1,
             'Unnamed': 2,
             'Point 3': 1,
             'Point 1': 1,
             'Spiculated region': 1,
             '': 1,
             'Espiculated Region': 1,
             'Calc': 1872})

In [17]:
name_2_name= {'CIRC': "Mass",
              'MISC': "Unknown",
              'ASYM': "Assymetry",
              'ARCH': "Distortion",
              'SPIC': "Spiculated Region",
              'CALC': "Calcification",
              'Calcification': "Calcification",
              'Spiculated Region': "Spiculated Region",
              'Mass': "Mass",
              'Cluster': "Cluster",
              'Assymetry': "Assymetry",
              'Asymmetry': "Assymetry",
              'Distortion': "Distortion",
              'Calcifications': "Calcification",
              'Unnamed': "Unknown",
              'Point 3': "Unknown",
              'Point 1': "Unknown",
              'Spiculated region': "Spiculated Region",
              '': "Unknown",
              'Calc': "Calcification",
              'Espiculated Region': "Spiculated Region"}

In [18]:
def generate_yolo_box(bboxes, image_shape):
    bboxes = np.array(bboxes)
    #Calc the xcenter and ycenter. bboxes = [xmin, ymin, w, d]
    xcenter = bboxes[0] + bboxes[2]/2
    ycenter = bboxes[1] + bboxes[3]/2
    
    #Calc proportion to image
    xcenter = xcenter / image_shape[1]
    ycenter = ycenter / image_shape[0]
    w = bboxes[2] / image_shape[1]
    h = bboxes[3] / image_shape[0]
    return xcenter, ycenter, w, h   

In [21]:
from tensorflow.keras.utils import load_img

def load_img_from_path_no_resize(file_path):
    img = tf.io.read_file(file_path)
    img = tf.io.decode_image(img, channels=1)
    return img

def resize_correct_side(image):
    h_in = image.shape[0]
    w_in = image.shape[1]
    
    input_is_wider = h_in <= w_in
    output_is_wider = input_shape[0] <= input_shape[1]
    
    increase_width_first = abs(h_in/input_shape[0] - 1.0) < abs(w_in/input_shape[1] - 1.0)
    N = input_shape[1] if increase_width_first else input_shape[0]
    if input_is_wider != output_is_wider:
        return albu.LongestMaxSize(max_size=N)
    else:
        return albu.SmallestMaxSize(max_size=N)
    

def apply_albumentations(image, bboxes):
    image = image.astype("uint8")    
    
    transform = albu.Compose([
        resize_correct_side(image),
        albu.PadIfNeeded(min_height=input_shape[0], min_width=input_shape[1], border_mode=0, value=(0,0,0)),
        albu.CLAHE(clip_limit=(1,10),p=1),
        #albu.Rotate(p=1, border_mode=cv2.BORDER_CONSTANT,)
        ],
        bbox_params=albu.BboxParams(format='coco', label_fields=["abnormality"])
    )
    
    transform_data = transform(image = image, bboxes=bboxes, abnormality=[1]*len(bboxes))
    new_img = transform_data["image"]
    new_boxes = transform_data["bboxes"]
    return new_img.astype("uint8"), new_boxes

def check_type_abnormality(ab):
    return name_2_name[ab["type"]] in ["Calcification", "Mass"]

@tf.function
def preprocess_img_and_boxes(image):
    for file, roi in rois.items():
        roi = list(filter(lambda r: check_type_abnormality(r), roi))
        boxes=[]
        for box in roi:
            boxes.append([box["x"], box["y"], box["w"], box["h"]])
    image, boxes = tf.numpy_function(apply_albumentations, inp=[image, boxes], Tout=[tf.uint8, tf.double])
    return image, boxes

ds = ds_all_files.map(load_img_from_path_no_resize, num_parallel_calls=tf.data.AUTOTUNE)
ds = ds.map(preprocess_img_and_boxes, num_parallel_calls=tf.data.AUTOTUNE)

In [24]:
YOLO_DIR_TEST="/kaggle/working/test"
YOLO_DIR_TEST_FILE="/kaggle/working/test_files"

os.makedirs(YOLO_DIR_TEST, exist_ok=True)
os.makedirs(YOLO_DIR_TEST_FILE, exist_ok=True)

counter=0
for image, label in ds:
    #plt.imshow(image.numpy())
    #print("Label: ", label.numpy())
    denoised_image=model.predict(np.array(image).reshape(input_shape), axes="YX")
    visualize(denoised_image, label,[1]*len(label),{1: "Abnormality"})
    
    cv2.imwrite(f"{os.path.join(YOLO_DIR_TEST, f'{counter}.png')}", denoised_image)
    with open(os.path.join(YOLO_DIR_TEST_FILE, f'{counter}.txt'),"w") as f:
        for box in label.numpy():
            yolo_box = generate_yolo_box(box, denoised_image.shape)
            f.write(f"Abnormality {yolo_box[0]} {yolo_box[1]} {yolo_box[2]} {yolo_box[3]} \n")
    counter+=1

The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 59ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 22ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 28ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 21ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 

1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 17ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 17ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 22ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint

1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 28ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 17ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint

1/1 [==============================] - 0s 18ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 21ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 27ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 26ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 24ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint

1/1 [==============================] - 0s 21ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 21ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 26ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 22ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 22ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 24ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 21ms/step
The input image is of type uint

...

1/1 [==============================] - 0s 21ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 22ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 21ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 21ms/step
The input image is of type uint

1/1 [==============================] - 0s 28ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 25ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 28ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 21ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 22ms/step
The input image is of type uint

1/1 [==============================] - 0s 22ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 22ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 25ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 27ms/step
The input image is of type uint

1/1 [==============================] - 0s 21ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 22ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 22ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 26ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 37ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 26ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 27ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 27ms/step
The input image is of type uint

1/1 [==============================] - 0s 25ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 27ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 24ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 33ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 34ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 22ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 21ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint